# ucr_basic_test

In [1]:
import pandas as pd
import numpy as np

# read data
data = pd.read_csv("UCR_agency_count_raw.csv")

# print all columns
print(data.columns)

data.shape

Index(['ori', 'ori9', 'agency_name', 'year', 'month', 'state',
       'country_division', 'fips_state_code', 'fips_county_code'],
      dtype='object')


(3530856, 9)

In [2]:
# process
agency_counts = data.groupby(['state', 'year'])['ori'].nunique().reset_index().rename(columns={'ori':'count'})

agency_counts.head(150)

,state,year,count
0,alabama,2000,419
1,alabama,2001,424
2,alabama,2002,426
3,alabama,2003,430
4,alabama,2004,434
...,...,...,...
145,florida,2005,1081
146,florida,2006,1081
147,florida,2007,1083
148,florida,2008,1085


In [4]:
# check whether number of agencies in each state is monotone
from pandas.api.types import is_numeric_dtype
from typing import Union

def check_monotonicity(series:pd.Series) -> Union[bool, str]:
    if is_numeric_dtype(series):
        return series.is_monotonic_increasing
    else:
        return 'n/a'

monotonicity = agency_counts.groupby('state')['count'].apply(lambda s: check_monotonicity(s))

monotonicity.rename('increasing')


    

state
alabama True
alaska True
arizona True
arkansas True
california True
canal zone True
colorado True
connecticut True
delaware True
district of columbia True
florida True
georgia True
guam True
hawaii True
idaho True
illinois True
indiana True
iowa True
kansas True
kentucky True
louisiana True
maine True
maryland True
massachusetts True
michigan True
minnesota True
mississippi True
missouri True
montana True
nebraska True
nevada True
new hampshire True
new jersey True
new mexico True
new york True
north carolina True
north dakota True
ohio True
oklahoma True
oregon True
pennsylvania True
puerto rico True
rhode island True
south carolina True
south dakota True
tennessee True
texas True
utah True
vermont True
virgin islands True
virginia True
washington True
west virginia True
wisconsin True
wyoming True
Name: increasing, dtype: bool

In [11]:
# generate multipliers

# s = pd.Series([1,2,3,6])

# s.div(s.shift(-1)).fillna(1)

def get_product(s:pd.Series) -> pd.Series:
    product = []
    for i in range(0, s.size):
        if i == 0:
            product.append(1)
        else:
            product.append(product[i-1] * s.iloc[i])
    return pd.Series(product)

# ascending
agency_counts['asc'] = agency_counts.groupby('state')['count'].transform(lambda s: s.div(s.shift(1)).fillna(1))
# descending
agency_counts['dsc'] = agency_counts.groupby('state')['count'].transform(lambda s: s.div(s.shift(-1)).fillna(1))
# ascending, deflators
agency_counts['asc_deflators'] = agency_counts.groupby('state')['asc'].apply(lambda s: get_product(s)).reset_index(drop=True)

agency_counts.head(100)

agency_counts.to_csv('changes_num_agencies.csv')


# a = pd.Series([1,2,3,4,5])
# get_product(a)

In [38]:
# aggregate changes in number of reporting agencies
# two types of changes: absolute and relative

# absolute diff
absolute_change = agency_counts.groupby('state')['count'].apply(lambda s: s.iloc[-1] - s.iloc[0])

relative_change = agency_counts.groupby('state')['count'].apply(lambda s: s.iloc[-1] / s.iloc[0])
print(absolute_change.idxmax(), absolute_change.max())
print(relative_change.idxmax(), relative_change.max())
print(absolute_change.sum() / absolute_change.size)
print(relative_change.sum() / relative_change.size)

missouri 355
missouri 2.028985507246377
46.27272727272727
1.1300425016814157


In [3]:
# investigate whether initial set of agencies preserve

initial_agency = data[['state', 'year', 'ori']]

unique_agency = initial_agency.groupby(['state', 'year']).agg({'ori':'unique'}).reset_index()

# unique_agency = initial_agency.drop_duplicates().reset_index()

unique_agency.head(100)

s_1 = unique_agency.loc[(unique_agency['year'] == 2000) & (unique_agency['state'] == 'alabama'), 'ori']
s_11 = pd.Series(['haha'], name='ori')
s_2 = unique_agency.loc[(unique_agency['year'] == 2001) & (unique_agency['state'] == 'alabama'), 'ori']
# unique_agency


# need a function to operate on a series of series.

# # np.isin(s_1, s_2, assume_unique=True)
# s_1 = pd.concat([s_1, s_11])
# s_1.explode()

# s = s_1.explode().isin(s_2.explode()).reset_index(drop=True)
# s
# # ~s.any()

def is_contained(s1:pd.Series, s2:pd.Series) -> bool:
    yes = s1.explode().isin(s2.explode()).reset_index(drop=True)
    return yes.all()

# def get_series(ss:list, index:int) -> (pd.Series, pd.Series):
#     """get exploded series for comparison."""

#     pass

def check_serial_subset(ss:pd.Series) -> bool:
    """Check whether elements in a pandas series satisfy the serial subset property."""
    ss.reset_index(drop=True)
    agency_pairs = [(ss.iloc[i], ss.iloc[i+1]) for i in range(ss.size - 1)]
    # print((agency_pairs[1])[1])
    truth_list = [is_contained(pd.Series(p[0]), pd.Series(p[1])) for p in agency_pairs]
    return all(truth_list)


# the driver program
serial_subset_test = unique_agency.groupby(['state'])['ori'].apply(lambda s: check_serial_subset(s))
serial_subset_test

# the serial subset property is satisfied

state
alabama True
alaska True
arizona True
arkansas True
california True
canal zone True
colorado True
connecticut True
delaware True
district of columbia True
florida True
georgia True
guam True
hawaii True
idaho True
illinois True
indiana True
iowa True
kansas True
kentucky True
louisiana True
maine True
maryland True
massachusetts True
michigan True
minnesota True
mississippi True
missouri True
montana True
nebraska True
nevada True
new hampshire True
new jersey True
new mexico True
new york True
north carolina True
north dakota True
ohio True
oklahoma True
oregon True
pennsylvania True
puerto rico True
rhode island True
south carolina True
south dakota True
tennessee True
texas True
utah True
vermont True
virgin islands True
virginia True
washington True
west virginia True
wisconsin True
wyoming True
Name: ori, dtype: bool

In [14]:
agencies_final = unique_agency.query('year == 2000').drop(columns='year').explode('ori').reset_index(drop=True).rename(columns={'ori':'agency'})
                 
agencies_final.to_csv('agency_to_keep.csv')
agencies_final

,state,agency
0,alabama,AL00100
1,alabama,AL00102
2,alabama,AL00103
3,alabama,AL00105
4,alabama,AL00107
...,...,...
19650,wyoming,WY01802
19651,wyoming,WY01803
19652,wyoming,WY01905
19653,wyoming,WY02302


# ucr_full_test

In [1]:
import pandas as pd
import numpy as np

# read data
data = pd.read_csv("UCR_testing.csv")

# print all columns
print(data.columns)

Index(['ori', 'population', 'year', 'month', 'state', 'population_group',
       'country_division', 'fips_state_code', 'fips_county_code',
       'address_city', 'number_of_months_reported'],
      dtype='object')


In [16]:
# number of observation and columns

data.shape

# the difference with UCR_agency_count_raw is that the observations with missing addres_city have been removed.

(3525324, 10)

In [17]:
# check whether relationship between city and agency is one-to-one
# also check: agency and county, city and county

# city and agency
t1 = data.groupby(['state', 'year', 'month', 'address_city'])['ori'].nunique().max() == 1

t2 = data.groupby(['state', 'year', 'month', 'ori'])['address_city'].nunique().max() == 1

print(t1, t2)

# this says that each agency can only correspond to one city

# agency and county
t1 = data.groupby(['state', 'year', 'month', 'fips_county_code'])['ori'].nunique().max() == 1

t2 = data.groupby(['state', 'year', 'month', 'ori'])['fips_county_code'].nunique().max() == 1

print(t1, t2)

# city and county
t1 = data.groupby(['state', 'year', 'month', 'fips_county_code'])['address_city'].nunique().max() == 1

t2 = data.groupby(['state', 'year', 'month', 'address_city'])['fips_county_code'].nunique().max() == 1

print(t1, t2)


False True
False True
False False


In [11]:
# does address_city satisfies the subseting property?

city_counts = data.groupby(['state', 'year'])['address_city'].nunique().reset_index().rename(columns={'address_city':'city_count'})

city_counts.head(100)

,state,year,city_count
0,ALABAMA,2000,312
1,ALABAMA,2001,314
2,ALABAMA,2002,315
3,ALABAMA,2003,317
4,ALABAMA,2004,319
...,...,...,...
95,CONNECTICUT,2011,103
96,CONNECTICUT,2012,103
97,CONNECTICUT,2013,103
98,DELAWARE,2000,42


In [14]:
# check whether number of agencies in each state is monotone

from pandas.api.types import is_numeric_dtype
from typing import Union

def check_monotonicity(series:pd.Series) -> Union[bool, str]:
    if is_numeric_dtype(series):
        return series.is_monotonic_increasing
    else:
        return 'n/a'

monotonicity = city_counts.groupby('state')['city_count'].apply(lambda s: check_monotonicity(s))

monotonicity.rename('increasing')

monotonicity.shape

(51,)

In [26]:
# population is the amount of people that an agency covers. Since UCR does not have full overage, the proepr aggregation of
# this variable should be less than state population estimate

# collapse population
# no multiple population values at the agency level
population_unique = data.groupby(['state', 'year', 'address_city', 'ori'])['population'].unique().reset_index()

population_unique['population'] = population_unique['population'].transform(lambda x: x[0])

population_unique.head(100)

,state,year,address_city,ori,population
0,ALABAMA,2000,abbeville,AL03700,9242
1,ALABAMA,2000,abbeville,AL03701,2987
2,ALABAMA,2000,adamsville,AL00122,4965
3,ALABAMA,2000,addison,AL06703,723
4,ALABAMA,2000,alabaster,AL05904,22619
...,...,...,...,...,...
95,ALABAMA,2000,cullman,AL02501,13995
96,ALABAMA,2000,cullman,AL02504,1966
97,ALABAMA,2000,dadeville,AL06200,19204
98,ALABAMA,2000,dadeville,AL06202,3212


In [28]:
# aggregate population at the city level

population_city = population_unique.groupby(['state', 'year', 'address_city'])['population'].apply('sum').reset_index()

population_city.head(100)

,state,year,address_city,population
0,ALABAMA,2000,abbeville,12229
1,ALABAMA,2000,adamsville,4965
2,ALABAMA,2000,addison,723
3,ALABAMA,2000,alabaster,22619
4,ALABAMA,2000,albertville,17247
...,...,...,...,...
95,ALABAMA,2000,fairfield,12381
96,ALABAMA,2000,fairhope,12480
97,ALABAMA,2000,falkville,1202
98,ALABAMA,2000,faunsdale,87


In [31]:
# aggregate at the state level

population_final = population_city.groupby(['state', 'year'])['population'].apply('sum').reset_index()

# save as csv file
population_final.to_csv('ucr_state_year_population.csv')

population_final.head(100)

,state,year,population
0,ALABAMA,2000,4483775
1,ALABAMA,2001,4502059
2,ALABAMA,2002,4516995
3,ALABAMA,2003,4531274
4,ALABAMA,2004,4550680
...,...,...,...
95,CONNECTICUT,2011,3672721
96,CONNECTICUT,2012,3682267
97,CONNECTICUT,2013,3687931
98,DELAWARE,2000,805723


# ucr_report_fully

In [3]:
data['num_month'] = data.groupby(['state', 'address_city', 'ori'])['number_of_months_reported'].transform(lambda x: x.nunique())

data.head(500)

,ori,population,year,month,state,population_group,country_division,fips_state_code,fips_county_code,address_city,number_of_months_reported,num_month
0,AK00101,260283,2000,april,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
1,AK00101,260283,2000,august,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
2,AK00101,260283,2000,december,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
3,AK00101,260283,2000,february,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
4,AK00101,260283,2000,january,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,AL04600,11015,2000,december,ALABAMA,"non-msa county 10,000 thru 24,999",east south central,1,87,tuskegee,1,2
496,AL04601,11846,2000,december,ALABAMA,"city 10,000 thru 24,999",east south central,1,87,tuskegee,1,2
497,AL04602,916,2000,december,ALABAMA,"city under 2,500",east south central,1,87,notasulga,1,2
498,AL04606,355,2000,december,ALABAMA,"city under 2,500",east south central,1,87,shorter,1,2


In [21]:
data[data['ori'] == 'AK00150']

,ori,population,year,month,state,population_group,country_division,fips_state_code,fips_county_code,address_city,number_of_months_reported,num_month
296,AK00150,532,2000,april,ALASKA,"city under 2,500",pacific,2,16,st. paul island,12,7
297,AK00150,532,2000,august,ALASKA,"city under 2,500",pacific,2,16,st. paul island,12,7
298,AK00150,532,2000,december,ALASKA,"city under 2,500",pacific,2,16,st. paul island,12,7
299,AK00150,532,2000,february,ALASKA,"city under 2,500",pacific,2,16,st. paul island,12,7
300,AK00150,532,2000,january,ALASKA,"city under 2,500",pacific,2,16,st. paul island,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...
3425038,AK00150,496,2013,august,ALASKA,"city under 2,500",pacific,2,16,st. paul island,0,7
3425039,AK00150,496,2013,september,ALASKA,"city under 2,500",pacific,2,16,st. paul island,0,7
3425040,AK00150,496,2013,october,ALASKA,"city under 2,500",pacific,2,16,st. paul island,0,7
3425041,AK00150,496,2013,november,ALASKA,"city under 2,500",pacific,2,16,st. paul island,0,7


In [18]:
# double check the num_months column

data['num_month'].nunique()

12

In [37]:
# total number of agencies
data['ori'].nunique()

22017

In [25]:
# total number of agencies that do not change month of reporting since start
data[data['num_month'] == 1]['ori'].nunique()

9239

In [36]:
# total number of agencies that 1) do not change month of reporting since start and 2) report 12 months fully

data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12)]['ori'].nunique()

3624

In [5]:
# the number of agencies that 1) meet the above criteria, 2) start reporting at 2000

data['num_year'] = data.groupby(['state', 'address_city', 'ori'])['year'].transform(lambda x: x.nunique())

data.head(100)

data['num_year'].unique()

data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12) & (data['num_year'] == 14)]['ori'].nunique()

3512

In [8]:
# for devent studies, use this reduced set of agencies

event_agency = data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12) & (data['num_year'] == 14)].groupby('state').agg({'ori':'unique'}).reset_index()

event_agency.explode('ori').reset_index(drop=True).to_csv('unique_agency_event.csv')

In [9]:
# now genreate a similar list of unique agencies for the full model (i.e., 2008-2013)
# do this for both yearly and monthly

data = data[data['year'] >= 2008]

data['num_month'] = data.groupby(['state', 'address_city', 'ori'])['number_of_months_reported'].transform(lambda x: x.nunique())

data.head(500)

,ori,population,year,month,state,population_group,country_division,fips_state_code,fips_county_code,address_city,number_of_months_reported,num_month
1958856,AK00101,280068,2008,april,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
1958857,AK00101,280068,2008,august,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
1958858,AK00101,280068,2008,december,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
1958859,AK00101,280068,2008,february,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
1958860,AK00101,280068,2008,january,ALASKA,"city 250,000 thru 499,999",pacific,2,20,anchorage,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1959351,AL02000,21384,2008,december,ALABAMA,"msa-county 10,000 thru 24,999",east south central,1,33,tuscumbia,1,2
1959352,AL02001,9118,2008,december,ALABAMA,"city 2,500 thru 9,999",east south central,1,33,sheffield,1,2
1959353,AL02002,8275,2008,december,ALABAMA,"city 2,500 thru 9,999",east south central,1,33,tuscumbia,1,2
1959354,AL02003,12925,2008,december,ALABAMA,"city 10,000 thru 24,999",east south central,1,33,muscle shoals,1,2


In [10]:
# total number of agencies
data['ori'].nunique()

22017

In [11]:
# total number of agencies that do not change month of reporting since start
data[data['num_month'] == 1]['ori'].nunique()

12882

In [12]:
# total number of agencies that 1) do not change month of reporting since start and 2) report 12 months fully

data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12)]['ori'].nunique()

6201

In [16]:
# the number of agencies that 1) meet the above criteria, 2) start reporting at 2000

data['num_year'] = data.groupby(['state', 'address_city', 'ori'])['year'].transform(lambda x: x.nunique())

print(data.head(100))

print(data['num_year'].unique())

data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12) & (data['num_year'] == 6)]['ori'].nunique()

             ori  population  year     month   state  \
1958856  AK00101      280068  2008     april  ALASKA   
1958857  AK00101      280068  2008    august  ALASKA   
1958858  AK00101      280068  2008  december  ALASKA   
1958859  AK00101      280068  2008  february  ALASKA   
1958860  AK00101      280068  2008   january  ALASKA   
...          ...         ...   ...       ...     ...   
1958951  AK00111        1998  2008      july  ALASKA   
1958952  AK00111        1998  2008      june  ALASKA   
1958953  AK00111        1998  2008     march  ALASKA   
1958954  AK00111        1998  2008       may  ALASKA   
1958955  AK00111        1998  2008  november  ALASKA   

                  population_group country_division  fips_state_code  \
1958856  city 250,000 thru 499,999          pacific                2   
1958857  city 250,000 thru 499,999          pacific                2   
1958858  city 250,000 thru 499,999          pacific                2   
1958859  city 250,000 thru 499,999     

6158

In [19]:
agg_agency = data[(data['num_month'] == 1) & (data['number_of_months_reported'] == 12) & (data['num_year'] == 6)].groupby('state').agg({'ori':'unique'}).reset_index()

agg_agency.explode('ori').reset_index(drop=True).to_csv('unique_agency_agg.csv')

agg_agency.head(100)

,state,ori
0,ALABAMA,[AL00112]
1,ALASKA,"[AK00101, AK00102, AK00103, AK00105, AK00108, ..."
2,ARIZONA,"[AZ00101, AZ00200, AZ00213, AZ00300, AZ00307, ..."
3,ARKANSAS,"[AR00102, AR00201, AR00202, AR00300, AR00400, ..."
4,CALIFORNIA,"[CA00100, CA00101, CA00102, CA00103, CA00104, ..."
5,COLORADO,"[CO00100, CO00101, CO00102, CO00103, CO00104, ..."
6,CONNECTICUT,"[CT00002, CT00004, CT00007, CT00009, CT00011, ..."
7,DELAWARE,"[DE00101, DE00102, DE00103, DE00104, DE00106, ..."
8,DISTRICT OF COLUMBIA,[DCMTP00]
9,GEORGIA,"[GA00500, GA00501, GA00503, GA00700, GA00701, ..."
